Nous allons entraîner un classifieur pour distinguer deux familles de protéines : 	ABC transporter (PF00005) et 	SH2 domain (PF00017).


1. Charger le fichier trainSeed.csv dans un data frame, la
première colonne contient les séquences et la deuxième leur classe PF00017 et PF00005.
Puis séparer les séquences et les classes en créant deux objets
`Xseqs` et `Y`. Xseqs contient les séquences en acid amine, `Y` l'identifiant Pfam. Dans `Y` nous pouvons remplacer PF00017 par 0 et PF00005 par 1.

In [18]:
import numpy as np
import pandas as pd

In [21]:
df = pd.read_csv('trainSeed.csv')
# print(df)

Xseqs = df['Sequence'].values
Y = (df['family'].values == 'PF00005').astype(int)

2. Nous allons utiliser la bibliothèque [protlearn](https://protlearn.readthedocs.io/en/latest/feature_extraction.html) pour extraire des features qui décrivent nos séquences protéiques. Utiliser la méthode ngram (n=2) pour extraire les fréquences de répétition de dinucléotides pour chaque sequence de `Xseqs`.

In [22]:
!pip install protlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 2.7 MB/s eta 0:00:00
  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/75/95/1917ac1ac6de32a087734833e7ecd5631f102cc35020daa2bee8558eca89/scikit_learn-1.4.0-1-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/03/e6/4aef6799badc2693548559bad5b56d56cfe89eada337c815fdfe92175250/xgboost-2.0.3-py3-none-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for mlxtend from https://files.pythonhosted.org/packages/1c/07/512f6a780239ad6ce06ce2aa7b4067583f5ddcfc7703a964a082c706a070/mlxtend-0.23.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 4.1 MB/s eta 0:00:0000:0100:01m
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for joblib>=0.13.2 from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a

In [37]:
from protlearn.features import ngram

X, _ = ngram(list(Xseqs), n=2)

3. Pour entraîner un modèle ML avec [sklearn](https://scikit-learn.org/stable/index.html) nous avons besoin de deux objets ndArray: `X` (de dimensions (NxM) qui comporte les `M` features de chaque protéine) et `y` (de dimension (Nx1) qui contient leurs classes), où N est le nombre total de samples (protéines) dans le dataset. Créer ces deux objets à partir des données chargées précédemment.

In [40]:
# X.shape = (95, 400); y.shape = (95,)

print(f'{X.shape=} {Y.shape=}')

X.shape=(95, 400) Y.shape=(95,)


4. Créer un modèle ML du type KNN (k=3), et juste pour avoir un aperçu de sa performance calculer son score sur les données d'entrainement X.

In [41]:
from sklearn.neighbors import KNeighborsClassifier

c = KNeighborsClassifier(n_neighbors=3)
c.fit(X, Y)
c.score(X, Y)

#score sur tout = 0.8210526315789474

0.8210526315789474

5. Diviser les données en deux parties, une pour l'entraînement et une pour le test. Utiliser `random_state=5` et `test_size=0.2`.
Entraîner le modèle K-NN (k= 3) sur les données d'entraînement et mesurer sa performance sur les données de test. Comparer les scores obtenu sur les donneées de test et sur le données de training.


In [46]:
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(X, Y, random_state=5, test_size=0.2)

c = KNeighborsClassifier(n_neighbors=3)
c.fit(trainX, trainY)
c.score(trainX, trainY), c.score(testX, testY)

#score sur training = 0.7894736842105263
#score sur test = 0.42105263157894735

(0.7894736842105263, 0.42105263157894735)

6. Une étape importante avant d'entraîner les modèles ML, est la normalisation des données. Normaliser les données en utilisant la méthode minmax puis recalculer le score sur les données de test ? Discutez vos résultats.


In [48]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
trainX_scaled = scaler.fit_transform(trainX)
testX_scaled = scaler.transform(testX)

c = KNeighborsClassifier(n_neighbors=3)
c.fit(trainX_scaled, trainY)
score_normalized = c.score(testX_scaled, testY)
print(f"Score on normalized test data: {score_normalized}")

#score sur test normalisé = 0.47368421052631576

Score on normalized test data: 0.47368421052631576


7. Créer un pipeline pour normaliser, entraîner et tester le modèle en même temps. Normaliser avec la méthode StandardScaler.
Afficher le score sur le jeu de test et expliquez la différence entre les deux
méthodes de scaling.

In [49]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

pipe = make_pipeline(StandardScaler(), KNeighborsClassifier(n_neighbors=3))

pipe.fit(trainX, trainY)
score_pipe = pipe.score(testX, testY)
print(f"Score on standardized test data: {score_pipe}")


Score on standardized test data: 0.5263157894736842


8. Afin de découvrir les meilleurs hyper-paramètres de votre modèle, utilisez la stratégie GridSearchCV. Faire varier les nombres de voisins entre 1 et 10, et utiliser  les distances euclidean et cosine. Quels sont les meilleurs paramètres? Sauvegarder le meilleur modèle.

In [56]:
import numpy as np
from sklearn.model_selection import GridSearchCV

g = GridSearchCV(KNeighborsClassifier(), {'n_neighbors': np.arange(1, 10), 'metric': ['euclidean',  'cosine']}, cv=5)
g.fit(trainX, trainY)
g.best_params_, g.best_score_

({'metric': 'cosine', 'n_neighbors': 2}, 0.9608333333333334)

9. Montrer les erreurs plus fréquentes de votre modèle à l'aide d'une matrice de confusion.

In [57]:
from sklearn.metrics import confusion_matrix

g.score(testX, testY), confusion_matrix(testY, g.predict(testX))

(1.0,
 array([[13,  0],
        [ 0,  6]]))

10. Discuter l'amélioration de modèle KNN, quelle conclusion tireriez-vous de ces expériences?

Les résultats sont bons cependant la taille de l'échantillon testé est vraiment petite il faudrait donc tester sur un plus grand échantillon.